In [ ]:
# Optimal loan payment schedule via MILP (pulp / CBC)
# Requires: pip install pulp pandas numpy

import pandas as pd
import numpy as np
import pulp
from pathlib import Path

# ---- User inputs / file ----
excel_path = Path("/Users/phillipsm/Documents/Professional/Loans/UpdatedNov2025/loans_info_no_mor.xlsx")   # change if needed
monthly_budget = 1000.0
T = 240   # planning horizon in months (adjust; must be large enough). Try 240 or 360.

# ---- Load & normalize loan table (adapt if your columns differ) ----
df = pd.read_excel(excel_path)
loans = pd.DataFrame({
    "name": df["Lender/Description"].fillna(df["Loan Number"].astype(str)).astype(str),
    "P": pd.to_numeric(df["Principal Balance"], errors="coerce"),
    "apr_percent": pd.to_numeric(df["Annual Interest Rate (%)"], errors="coerce"),
    "min_pay": pd.to_numeric(df["Minimum Monthly Payment"], errors="coerce")
})
loans["r"] = loans["apr_percent"] / 100.0 / 12.0   # monthly rate (decimal)
N = len(loans)

# Quick sanity
if monthly_budget < loans["min_pay"].sum():
    raise ValueError("Monthly budget is less than sum of minimums.")

# ---- MILP model ----
model = pulp.LpProblem("min_interest_payments", pulp.LpMinimize)

# Variables: bal_{i,t} for t=0..T, pay_{i,t} for t=1..T, z_{i,t} binary for t=1..T
bal = {}
pay = {}
z = {}
for i in range(N):
    for t in range(0, T+1):
        bal[(i,t)] = pulp.LpVariable(f"bal_{i}_{t}", lowBound=0, cat="Continuous")
    for t in range(1, T+1):
        pay[(i,t)] = pulp.LpVariable(f"pay_{i}_{t}", lowBound=0, cat="Continuous")
        z[(i,t)]   = pulp.LpVariable(f"z_{i}_{t}", lowBound=0, upBound=1, cat="Binary")

# initial balances
for i in range(N):
    model += bal[(i,0)] == float(loans.loc[i, "P"])

# Big-M choices
# balance upper bound M_bal_i: pick the initial principal (safe because we never let balance exceed this by model dynamics if paying)
M_bal = loans["P"].values * 5.0  # make a bit larger just in case; or use P * (1+r)**T
M_pay = np.minimum(monthly_budget, loans["P"].values * 5.0)  # can't pay more than monthly budget

# dynamics and constraints
for t in range(1, T+1):
    # monthly budget
    model += pulp.lpSum(pay[(i,t)] for i in range(N)) <= monthly_budget

    for i in range(N):
        r_i = float(loans.loc[i, "r"])
        m_i = float(loans.loc[i, "min_pay"])
        # balance update: bal_{t} = (1+r)*bal_{t-1} - pay_{t}
        model += bal[(i,t)] == (1.0 + r_i) * bal[(i,t-1)] - pay[(i,t)]

        # link z and balance: if bal_{i,t-1} > 0 then z must be 1; enforce bal_{i,t-1} <= M_bal_i * z_{i,t}
        model += bal[(i,t-1)] <= M_bal[i] * z[(i,t)]

        # min payment if active, and upper bound on pay if not active
        model += pay[(i,t)] >= m_i * z[(i,t)]
        model += pay[(i,t)] <= M_pay[i] * z[(i,t)]

        # Optional: do not allow payment greater than available outstanding (1+r)*bal_{t-1}
        # This is implied by balancing and bal>=0 but added for numerical safety:
        model += pay[(i,t)] <= (1.0 + r_i) * bal[(i,t-1)]

# Optional: ensure final balances are 0 (if you want full payoff within T). 
# If you want the solver to decide T automatically, set T large and require bal_{i,T} == 0
for i in range(N):
    model += bal[(i, T)] == 0

# Objective: minimize total interest = sum_t sum_i r_i * bal_{i,t-1}
model += pulp.lpSum(float(loans.loc[i, "r"]) * bal[(i, t-1)] for i in range(N) for t in range(1, T+1))

# ---- Solve ----
# Choose solver; pulp's default CBC is used here.
solver = pulp.PULP_CBC_CMD(msg=True, timeLimit=300)  # increase timeLimit if needed
res = model.solve(solver)

print("Status:", pulp.LpStatus[model.status])

# ---- Extract results ----
# Build per-month tables
rows_pay = []
rows_bal = []
monthly_totals = []

for t in range(1, T+1):
    month_principal_total = 0
    month_interest_total = 0
    
    for i in range(N):
        pay_val = pay[(i,t)].value() or 0.0
        bal_prev_val = bal[(i,t-1)].value() or 0.0
        r_i = float(loans.loc[i, "r"])
        
        # Interest accrued in month t is: r_i * bal[i,t-1]
        interest_val = r_i * bal_prev_val
        
        month_principal_total += pay_val
        month_interest_total += interest_val
        
        rows_pay.append({
            "month": t,
            "loan": loans.loc[i, "name"],
            "principal_payment": pay_val,
            "interest_accrued": interest_val,
            "total_payment": pay_val + interest_val
        })
        
    for i in range(N):
        rows_bal.append({
            "month": t,
            "loan": loans.loc[i, "name"],
            "balance": bal[(i,t)].value() or 0.0
        })
    
    monthly_totals.append({
        "month": t,
        "principal": month_principal_total,
        "interest": month_interest_total,
        "total_payment": month_principal_total + month_interest_total
    })

df_pay = pd.DataFrame(rows_pay)
df_bal = pd.DataFrame(rows_bal)
df_monthly = pd.DataFrame(monthly_totals)

out_dir = Path("/Users/phillipsm/Documents/Professional/Loans/UpdatedNov2025")
out_dir.mkdir(parents=True, exist_ok=True)
df_pay.to_csv(out_dir / "optimal_payments_by_month.csv", index=False)
df_bal.to_csv(out_dir / "balances_by_month.csv", index=False)
df_monthly.to_csv(out_dir / "monthly_totals.csv", index=False)

# summary: months until payoff (first month where all balances are zero)
payoff_month = None
for t in range(1, T+1):
    total_bal = sum((bal[(i,t)].value() or 0.0) for i in range(N))
    if total_bal < 0.01:  # accounting for floating point
        payoff_month = t
        break

print("Wrote outputs to", out_dir)
print("Objective (total interest):", pulp.value(model.objective))
print("Payoff month:", payoff_month)
print("\nMonthly summary (first 5 months):")
print(df_monthly.head())
